# 4. Inference

In [1]:
import sys, os
sys.path.append("../../..")

import pathSetting
import pandas as pd

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.common.common import p2_dataSelection as p2
from Clust.clust.ML.common.common import p4_testing as p4
from Clust.clust.ML.regression import test as p4R
from Clust.clust.ML.forecasting import p5_inference as p5

import torch
#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## 4-1. (just for test) Data Preparation 

In [2]:
# 0. pick only one data
DataMeta = p1.readJsonData(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())
# dataX

dataName_X = dataList[2]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[3]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

windowNum =144
dataX = p2.getSavedIntegratedData(dataSaveMode_X, dataName_X, dataFolderPath)[:windowNum]
datay = p2.getSavedIntegratedData(dataSaveMode_y, dataName_y, dataFolderPath)[:1]
from Clust.clust.transformation.type.DFToNPArray import transDFtoNP
inputX, inputy = transDFtoNP(dataX, datay, windowNum)
print(inputX, inputy) # Real Input

[[[40.26       40.26       40.29       ... 42.09       42.03
   41.9       ]
  [43.2        43.16333333 42.96333333 ... 42.36333333 42.43333333
   42.5       ]
  [38.53       38.5        38.63333333 ... 38.29       38.36333333
   38.29      ]
  ...
  [ 4.2         4.          3.8        ...  8.3         8.33333333
    8.36666667]
  [40.         40.         40.         ... 40.         40.
   40.        ]
  [ 4.          3.83333333  3.66666667 ...  7.          6.66666667
    6.33333333]]] [17.37000084]


In [3]:
dataX, datay

(                                RH_1       RH_2       RH_3       RH_4  \
 datetime                                                                
 2021-01-01 00:00:00+00:00  40.260000  43.200000  38.530000  37.363333   
 2021-01-01 00:10:00+00:00  40.260000  43.163333  38.500000  37.230000   
 2021-01-01 00:20:00+00:00  40.290000  42.963333  38.633333  37.200000   
 2021-01-01 00:30:00+00:00  40.290000  42.490000  38.790000  37.200000   
 2021-01-01 00:40:00+00:00  40.626667  42.156667  38.596667  37.090000   
 ...                              ...        ...        ...        ...   
 2021-01-01 23:10:00+00:00  41.500000  42.090000  38.290000  40.730000   
 2021-01-01 23:20:00+00:00  41.833333  42.290000  38.290000  40.790000   
 2021-01-01 23:30:00+00:00  42.090000  42.363333  38.290000  41.166667   
 2021-01-01 23:40:00+00:00  42.030000  42.433333  38.363333  42.093333   
 2021-01-01 23:50:00+00:00  41.900000  42.500000  38.290000  42.400000   
 
                                 RH_

In [4]:
inputX, inputy

(array([[[40.26      , 40.26      , 40.29      , ..., 42.09      ,
          42.03      , 41.9       ],
         [43.2       , 43.16333333, 42.96333333, ..., 42.36333333,
          42.43333333, 42.5       ],
         [38.53      , 38.5       , 38.63333333, ..., 38.29      ,
          38.36333333, 38.29      ],
         ...,
         [ 4.2       ,  4.        ,  3.8       , ...,  8.3       ,
           8.33333333,  8.36666667],
         [40.        , 40.        , 40.        , ..., 40.        ,
          40.        , 40.        ],
         [ 4.        ,  3.83333333,  3.66666667, ...,  7.        ,
           6.66666667,  6.33333333]]]),
 array([17.37000084]))

In [5]:
from Clust.clust.ingestion.mongo import mongoClient as mongo_client
from Clust.setting import influx_setting_KETI as ins

mongo_client = mongo_client.mongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [6]:
db_name = 'model'
collection_name = 'meta'
search = {'modelName': 'EnergyLSTMRegression'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
ModelMeta = model_meta[1]

In [7]:
ModelMeta

{'author': 'KETI',
 'modelName': 'EnergyLSTMRegression',
 'modelVersion': 0.1,
 'trainDataInfo': {'startTime': '2016-01-11',
  'endTime': '2016-04-15',
  'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'],
   ['life_indoor_environment', 'temperatureTrain_10min'],
   ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 600,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'featureList': ['RH_1',
  'RH_2',
  'RH_3',
  'RH_4',
  'RH_5',
  'RH_6',
  'RH_7',
  'RH_8',
  'RH_9',
  'T1',
  'T2',
  'T3',
  'T4',
  'T5',
  'T6',
  'T7',
  'T8',
  'T9',
  'Press_mm_hg',
  'RH_out',
  'T_out'

In [8]:
X_scalerFilePath = ModelMeta['files']['XScalerFile']["filePath"]
y_scalerFilePath = ModelMeta['files']['yScalerFile']["filePath"]
modelFilePath = ModelMeta['files']['modelFile']["filePath"]

featureList = ModelMeta["featureList"]
target = ModelMeta["target"]
scalerParam = ModelMeta["scalerParam"]
model_method = ModelMeta["model_method"]
trainParameter = ModelMeta["trainParameter"]


In [9]:
#2
# from Clust.clust.transformation.type.NPArrayToDF import trans3NPtoDF
# inputXDF, y = trans3NPtoDF(inputX, inputy,'2000-1-1')

In [10]:
# 3. Scaling
test_x, scaler_X = p4.getScaledTestData(dataX[featureList], X_scalerFilePath, scalerParam)
test_y, scaler_y = p4.getScaledTestData(datay[target], y_scalerFilePath, scalerParam)
# y  값은 더미 값이 들어가는 것임

In [11]:
test_x, scaler_X

(                               RH_1      RH_2      RH_3      RH_4      RH_5  \
 datetime                                                                      
 2021-01-01 00:00:00+00:00  0.364279  0.664944  0.456302  0.386490  0.275336   
 2021-01-01 00:10:00+00:00  0.364279  0.663872  0.454900  0.380531  0.275336   
 2021-01-01 00:20:00+00:00  0.365104  0.658023  0.461131  0.379190  0.274333   
 2021-01-01 00:30:00+00:00  0.365104  0.644180  0.468453  0.379190  0.274333   
 2021-01-01 00:40:00+00:00  0.374369  0.634432  0.459417  0.374274  0.274333   
 ...                             ...       ...       ...       ...       ...   
 2021-01-01 23:10:00+00:00  0.398404  0.632482  0.445085  0.536963  0.335480   
 2021-01-01 23:20:00+00:00  0.407577  0.638331  0.445085  0.539644  0.331521   
 2021-01-01 23:30:00+00:00  0.414641  0.640476  0.445085  0.556479  0.328413   
 2021-01-01 23:40:00+00:00  0.412990  0.642523  0.448512  0.597896  0.326960   
 2021-01-01 23:50:00+00:00  0.409412  0.

In [12]:
test_y, scaler_y

(                              value
 datetime                           
 2021-01-01 00:00:00+00:00  0.574499,
 MinMaxScaler())

In [13]:
# 4. Testing
df_result, result_metrics = p4R.get_result_metrics(trainParameter, test_x, test_y, model_method, target, modelFilePath, scalerParam, scaler_y, windowNum=0)


Start testing data

** Performance of test dataset ==> MSE = 0.07445332407951355, MAE = 0.2728613615036011
** Dimension of result for test dataset = (1,)
scale


../../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
../../../../Clust/clust/transformation/type/DFToNPArray.py:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]
/apptest/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [14]:
df_result

,value,prediction
datetime,,
2021-01-01 00:00:00+00:00,17.370001,11.656283


In [22]:
result_metrics

{'mae': 5.7137175,
 'rmse': 5.713717555002159,
 'r2': nan,
 'mape': 32.89417028427124}

In [15]:
ModelMeta['files']['modelFile']["filePath"]

'./Models/LSTM_rg/EnergyLSTMRegression/IntegraionTrainX/618f2ca7c380b25efe12bee2a2ab1869/model.pkl'